In [24]:
import os
import pickle
import numpy as np

from sklearn.model_selection import KFold

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
import torch.nn.functional as F


from networks.mlp import MLP


In [3]:
protein_seq = pickle.load(open('assignment1_data/d1a2pa_.pkl', 'rb'))
protein_seq

{'seq': 'AQVINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPGKSIGGDIFSNREGKLPGKSGRTWREADINYTSGFRNSDRILYSSDWLIYKTTDHYQTFTKIR',
 'ssp': 'CCCCCCHHHHHHHHHHHCCCCCCEECHHHHHHHCCCHHHCCHHHHCCCCEEEEEEECCCCCCCCCCCCCCEEEEECCCCCCCCCCCEEEEECCCCEEEECCCCCCCEECC'}

In [32]:
len(protein_seqs[2]['seq'])

135

In [8]:
protein_seqs = []

data_source = os.listdir('assignment1_data')
for file in data_source:
    if file.endswith('.pkl'):
        protein_seq = pickle.load(open('assignment1_data/' + file, 'rb'))
        protein_seqs.append(protein_seq)
print('file number:', len(data_source), '\tprotein number:', len(protein_seqs))

file number: 3000 	protein number: 3000


In [17]:
data = []
label = []

for sequence in protein_seqs:
    seq, ssp = sequence['seq'], sequence['ssp']
    data.append(seq)
    label.append(ssp)
print('data number:', len(data), '\tlabel number:', len(label))

data number: 3000 	label number: 3000


In [22]:
X = np.array(data)
y = np.array(label)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print('X_train:', X_train.shape, '\tX_test:', X_test.shape, '\ty_train:', y_train.shape, '\ty_test:', y_test.shape)
    # print('y_train:', y_train, '\ty_test:', y_test)
    # break

X_train: (2400,) 	X_test: (600,) 	y_train: (2400,) 	y_test: (600,)
X_train: (2400,) 	X_test: (600,) 	y_train: (2400,) 	y_test: (600,)
X_train: (2400,) 	X_test: (600,) 	y_train: (2400,) 	y_test: (600,)
X_train: (2400,) 	X_test: (600,) 	y_train: (2400,) 	y_test: (600,)
X_train: (2400,) 	X_test: (600,) 	y_train: (2400,) 	y_test: (600,)


In [27]:
batch_size = 32

class ProteinDataset(Dataset):
    
    def __init__(self, seq_dir):
        self.protein_seqs = []
        self.seq_dir = seq_dir
        data_source = os.listdir(seq_dir)
        for file in data_source:
            if file.endswith('.pkl'):
                protein_seq = pickle.load(open(os.path.join(seq_dir, file), 'rb'))
                self.protein_seqs.append(protein_seq)

    def __len__(self):
        return len(self.protein_seqs)

    def __getitem__(self, idx):
        sequence = self.protein_seqs[idx]
        seq, ssp = sequence['seq'], sequence['ssp']
        return seq, ssp
        

dataset = ProteinDataset(seq_dir='assignment1_data')

# for i in range(5):
num_train = len(dataset)
split = int(np.floor(0.2 * num_train))
indices = list(range(num_train))
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

def set_model():
    model = MLP()
    cristerion = F.cross_entropy()

    # enable synchronized Batch Normalization
    if opt.syncBN:
        model = apex.parallel.convert_syncbn_model(model)

    if torch.cuda.is_available():
        if torch.cuda.device_count() > 1:
            model.encoder = torch.nn.DataParallel(model.encoder)
        model = model.cuda()
        criterion = criterion.cuda()
        cudnn.benchmark = True

    return model, criterion
